In [4]:
%AddJar http://0.0.0.0:8888/edit/NYSK/lsa.jar

Starting download from http://0.0.0.0:8888/edit/NYSK/lsa.jar
Finished download of lsa.jar


Magic AddJar failed to execute with error: 
Jar 'lsa.jar' is not valid.

In [8]:
%AddJar http://cedric.cnam.fr/~ferecatu/RCP216/tp/tptexte/lsa.jar

Using cached version of lsa.jar


Magic AddJar failed to execute with error: 
Jar 'lsa.jar' is not valid.

In [3]:
import org.apache.spark.SparkContext
import org.apache.spark.SparkContext._
import org.apache.spark.SparkConf
import org.apache.spark.rdd.RDD
import org.apache.spark.rdd

import scala.xml._

import org.apache.hadoop.io.{ Text, LongWritable }
import org.apache.hadoop.conf.Configuration

import com.cloudera.datascience.common.XmlInputFormat
import com.cloudera.datascience.lsa.ParseWikipedia._
import com.cloudera.datascience.lsa.RunLSA._

import org.apache.spark.mllib.linalg._
import org.apache.spark.mllib.linalg.distributed.RowMatrix
import breeze.linalg.{DenseMatrix => BDenseMatrix, DenseVector => BDenseVector, SparseVector => BSparseVector, Vector => BVector}
import org.apache.spark.mllib.feature.{Word2Vec, Word2VecModel}
import org.apache.spark.mllib.feature.StandardScaler



import java.io.StringReader
import edu.stanford.nlp.ling.CoreLabel;
import edu.stanford.nlp.ling.HasWord;
import edu.stanford.nlp.process.CoreLabelTokenFactory;
import edu.stanford.nlp.process.DocumentPreprocessor;
import edu.stanford.nlp.process.PTBTokenizer;

import java.sql.Timestamp
import java.text.SimpleDateFormat

import org.apache.spark.mllib.clustering.{ KMeans, KMeansModel }
import org.apache.spark.mllib.util.KMeansDataGenerator

import scala.collection.mutable.HashMap


Name: Compile Error
Message: <console>:36: error: object cloudera is not a member of package com
       import com.cloudera.datascience.common.XmlInputFormat
                  ^
<console>:37: error: object cloudera is not a member of package com
       import com.cloudera.datascience.lsa.ParseWikipedia._
                  ^
<console>:38: error: object cloudera is not a member of package com
       import com.cloudera.datascience.lsa.RunLSA._
                  ^
<console>:49: error: object stanford is not a member of package edu
       import edu.stanford.nlp.ling.CoreLabel;
                  ^
<console>:50: error: object stanford is not a member of package edu
       import edu.stanford.nlp.ling.HasWord;
                  ^
<console>:51: error: object stanford is not a member of package edu
       import edu.stanford.nlp.process.CoreLabelTokenFactory;
                  ^
<console>:52: error: object stanford is not a member of package edu
       import edu.stanford.nlp.process.DocumentP

In [39]:
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.rdd.RDD

In [2]:
def toBreeze(v:Vector) = BVector(v.toArray)
def fromBreeze(bv:BVector[Double]) = Vectors.dense(bv.toArray)
def add(v1:Vector, v2:Vector) = fromBreeze(toBreeze(v1) + toBreeze(v2))
def scalarMultiply(a:Double, v:Vector) = fromBreeze(a * toBreeze(v))

def stackVectors(v1:Vector, v2:Vector) = {
var v3 = Vectors.zeros(v1.size+v2.size)
    for (i <- 0 until v1.size) {
      BVector(v3.toArray)(i) = v1(i);
    }
    for (i <- 0 until v2.size) {
      BVector(v3.toArray)(v1.size+i) = v2(i);
    }
}

// SÃ©paration du fichier XML en un RDD oÃ¹ chaque Ã©lÃ©ment est un article
// Retourne un RDD de String Ã  partir du fichier "path"
def loadArticle(sc: SparkContext, path: String): RDD[String] = {
@transient val conf = new Configuration()
conf.set(XmlInputFormat.START_TAG_KEY, "<document>")
conf.set(XmlInputFormat.END_TAG_KEY, "</document>")
val in = sc.newAPIHadoopFile(path, classOf[XmlInputFormat], classOf[LongWritable], classOf[Text], conf)
in.map(line => line._2.toString)
}


// Pour un élément XML de type "document",
//   - on extrait le champ "date"
//   - on parse la chaÃ®ne de caractÃ¨re au format yyyy-MM-dd HH:mm:ss
//   - on retourne un Timestamp
def extractDate(elem: scala.xml.Elem): java.sql.Timestamp = {
    val dn: scala.xml.NodeSeq = elem \\ "date"
    val x: String = dn.text
    // d'aprÃ¨s l'exemple 2011-05-18 16:30:35
    val format = new SimpleDateFormat("yyyy-MM-dd HH:mm:ss")
    if (x == "")
      return null
    else {
      val d = format.parse(x.toString());
      val t = new Timestamp(d.getTime());
      return t
    }
}

// Pour un élément XML de type "document",
//   - on extrait le champ #field
def extractString(elem: scala.xml.Elem, field: String): String = {
    val dn: scala.xml.NodeSeq = elem \\ field
    val x: String = dn.text
    return x
}

def extractInt(elem: scala.xml.Elem, field: String): Int = {
    val dn: scala.xml.NodeSeq = elem \\ field
    val x: Int = dn.text.toInt
    return x
}

def extractAll(elem: scala.xml.Elem, whatText: String = "text"): (Int, java.sql.Timestamp, String) = {
    return (extractInt(elem,"docid"), extractDate(elem), extractString(elem,whatText))
}

def extractText(elem: scala.xml.Elem): String = {
    return (extractString(elem,"title") + " " + extractString(elem,"summary") + " " + extractString(elem,"text"))
}

// NÃ©cessaire, car le type java.sql.Timestamp n'est pas ordonnÃ© par dÃ©faut (étonnant...)
implicit def ordered: Ordering[java.sql.Timestamp] = new Ordering[java.sql.Timestamp] {
def compare(x: java.sql.Timestamp, y: java.sql.Timestamp): Int = x compareTo y
}

def hasLetters(str: String): Boolean = {
// While loop for high performance
var i = 0
while (i < str.length) {
  if (Character.isLetter(str.charAt(i))) {
    return true
  }
  i += 1
}
false
}



Name: Compile Error
Message: <console>:27: error: not found: value BVector
def toBreeze(v:Vector) = BVector(v.toArray)
                         ^
<console>:27: error: type Vector takes type parameters
def toBreeze(v:Vector) = BVector(v.toArray)
               ^
<console>:28: error: not found: value Vectors
def fromBreeze(bv:BVector[Double]) = Vectors.dense(bv.toArray)
                                     ^
<console>:28: error: not found: type BVector
def fromBreeze(bv:BVector[Double]) = Vectors.dense(bv.toArray)
                  ^
<console>:29: error: type Vector takes type parameters
def add(v1:Vector, v2:Vector) = fromBreeze(toBreeze(v1) + toBreeze(v2))
           ^
<console>:29: error: type Vector takes type parameters
def add(v1:Vector, v2:Vector) = fromBreeze(toBreeze(v1) + toBreeze(v2))
                      ^
<console>:30: error: type Vector takes type parameters
def scalarMultiply(a:Double, v:Vector) = fromBreeze(a * toBreeze(v))
                               ^
<console>:33: 

In [41]:
    val stopwords = sc.textFile("stopwords.txt").collect.toArray.toSet
    val stopwordsBroadcast = sc.broadcast(stopwords).value

stopwords = Set(down, it's, that's, for, further, she'll, any, there's, this, haven't, in, ought, myself, have, your, off, once, i'll, are, is, his, why, too, why's, am, than, isn't, didn't, himself, but, you're, below, what, would, i'd, if, you'll, own, they'll, up, we're, they'd, so, our, do, all, him, ours	ourselves, had, nor, before, it, a, she's, as, hadn't, because, has, she, yours, or, above, yourself, herself, she'd, such, they, each, can't, don't, i, until, that, out, he's, cannot, to, we've, hers, you, did, let's, most, here, these, hasn't, was, there, when's, shan't, doing, at, through, been, over, i've, on, being, same, how, whom, my, after, who, itself, me, them, by, then, couldn't, he, should, few, wasn't, again, while, their, not, w...


Set(down, it's, that's, for, further, she'll, any, there's, this, haven't, in, ought, myself, have, your, off, once, i'll, are, is, his, why, too, why's, am, than, isn't, didn't, himself, but, you're, below, what, would, i'd, if, you'll, own, they'll, up, we're, they'd, so, our, do, all, him, ours	ourselves, had, nor, before, it, a, she's, as, hadn't, because, has, she, yours, or, above, yourself, herself, she'd, such, they, each, can't, don't, i, until, that, out, he's, cannot, to, we've, hers, you, did, let's, most, here, these, hasn't, was, there, when's, shan't, doing, at, through, been, over, i've, on, being, same, how, whom, my, after, who, itself, me, them, by, then, couldn't, he, should, few, wasn't, again, while, their, not, w...

In [42]:
import org.apache.spark.mllib.feature.Word2VecModel
// lire le Word2VecModel
val w2vModel = Word2VecModel.load(sc, "w2vModel")

java.lang.AssertionError: assertion failed: 
  Object
        with Serializable {
  def $init$(): Unit
  def size(): Int
  def toArray(): Array[Double]
  override def equals(other: Object): Boolean
  override def hashCode(): Int
  private[package spark] def asBreeze(): breeze.linalg.Vector
  def apply(i: Int): Double
  def copy(): org.apache.spark.mllib.linalg.Vector
  def foreachActive(f: Function2): Unit
  def numActives(): Int
  def numNonzeros(): Int
  def toSparse(): org.apache.spark.mllib.linalg.SparseVector
  private[package linalg] def toSparseWithSize(nnz: Int): org.apache.spark.mllib.linalg.SparseVector
  def toDense(): org.apache.spark.mllib.linalg.DenseVector
  def compressed(): org.apache.spark.mllib.linalg.Vector
  def argmax(): Int
  def toJson(): String
  def asML(): org.apache.spark.ml.linalg.Vector
}
     while compiling: <console>
        during phase: globalPhase=terminal, enteringPhase=jvm
     library version: version 2.11.12
    compiler version: version 2.11.12


Name: java.lang.RuntimeException
Message: error reading Scala signature of org.apache.spark.mllib.feature.Word2VecModel: assertion failed: 
  Object
        with Serializable {
  def $init$(): Unit
  def size(): Int
  def toArray(): Array[Double]
  override def equals(other: Object): Boolean
  override def hashCode(): Int
  private[package spark] def asBreeze(): breeze.linalg.Vector
  def apply(i: Int): Double
  def copy(): org.apache.spark.mllib.linalg.Vector
  def foreachActive(f: Function2): Unit
  def numActives(): Int
  def numNonzeros(): Int
  def toSparse(): org.apache.spark.mllib.linalg.SparseVector
  private[package linalg] def toSparseWithSize(nnz: Int): org.apache.spark.mllib.linalg.SparseVector
  def toDense(): org.apache.spark.mllib.linalg.DenseVector
  def compressed(): org.apache.spark.mllib.linalg.Vector
  def argmax(): Int
  def toJson(): String
  def asML(): org.apache.spark.ml.linalg.Vector
}
     while compiling: <console>
        during phase: globalPhase=terminal,

In [43]:
val conf = new SparkConf().setAppName("NYSK").setMaster("local[*]").set("spark.driver.memory", "10g")



conf.set("spark.hadoop.validateOutputSpecs", "false")
//conf.set("spark.driver.allowMultipleContexts", "true")
//conf.setMaster("local[*]")
//conf.set("spark.executor.memory", MAX_MEMORY)
//conf.set("spark.driver.memory", MAX_MEMORY)
//conf.set("spark.driver.maxResultSize", MAX_MEMORY)
val sc = new SparkContext(conf)


val hadoopConf = new org.apache.hadoop.conf.Configuration()

lastException = null


Name: org.apache.spark.SparkException
Message: Only one SparkContext may be running in this JVM (see SPARK-2243). To ignore this error, set spark.driver.allowMultipleContexts = true. The currently running SparkContext was created at:
org.apache.spark.sql.SparkSession$Builder.getOrCreate(SparkSession.scala:926)
org.apache.toree.kernel.api.Kernel.sparkSession(Kernel.scala:444)
org.apache.toree.kernel.api.Kernel.sparkContext(Kernel.scala:449)
org.apache.toree.kernel.api.Kernel$$anonfun$addJars$3.apply(Kernel.scala:80)
org.apache.toree.kernel.api.Kernel$$anonfun$addJars$3.apply(Kernel.scala:80)
scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
scala.collection.mutable.WrappedArray.foreach(WrappedArray.scala:35)
org.apache.toree.kernel.api.Kernel.addJars(Kernel.scala:80)
org.apache.toree.magic.builtin.AddJar.execute(AddJar.scala:181)
sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl

In [44]:
    var textToExtract = "text";
    var useW2Vec = true;
    var weightTfIdf = true;
    println ("*****");

*****


textToExtract = text
useW2Vec = true
weightTfIdf = true


lastException: Throwable = null


true

In [45]:
    val nysk_raw = loadArticle(sc, "nysk.xml")/*.sample(false,0.01)*/
    val nysk_xml: RDD[Elem] = nysk_raw.map(XML.loadString)

nysk_raw = MapPartitionsRDD[55] at map at <console>:180
nysk_xml = MapPartitionsRDD[56] at map at <console>:159


MapPartitionsRDD[56] at map at <console>:159

In [46]:
 val nysk: RDD[(Int, java.sql.Timestamp, String)] = nysk_xml.map(e => extractAll(e,textToExtract))
    val nyskTitles: RDD[(Int, java.sql.Timestamp, String)] = nysk_xml.map(e => extractAll(e,"title"))
    val nyskSummaries: RDD[(Int, java.sql.Timestamp, String)] = nysk_xml.map(e => extractAll(e,"summary"))

nysk = MapPartitionsRDD[57] at map at <console>:158
nyskTitles = MapPartitionsRDD[58] at map at <console>:159
nyskSummaries = MapPartitionsRDD[59] at map at <console>:160


MapPartitionsRDD[59] at map at <console>:160

In [47]:
    val stopwords = sc.textFile("stopwords.txt").collect.toArray.toSet
    val stopwordsBroadcast = sc.broadcast(stopwords).value

stopwords = Set(down, it's, that's, for, further, she'll, any, there's, this, haven't, in, ought, myself, have, your, off, once, i'll, are, is, his, why, too, why's, am, than, isn't, didn't, himself, but, you're, below, what, would, i'd, if, you'll, own, they'll, up, we're, they'd, so, our, do, all, him, ours	ourselves, had, nor, before, it, a, she's, as, hadn't, because, has, she, yours, or, above, yourself, herself, she'd, such, they, each, can't, don't, i, until, that, out, he's, cannot, to, we've, hers, you, did, let's, most, here, these, hasn't, was, there, when's, shan't, doing, at, through, been, over, i've, on, being, same, how, whom, my, after, who, itself, me, them, by, then, couldn't, he, should, few, wasn't, again, while, their, not, w...


Set(down, it's, that's, for, further, she'll, any, there's, this, haven't, in, ought, myself, have, your, off, once, i'll, are, is, his, why, too, why's, am, than, isn't, didn't, himself, but, you're, below, what, would, i'd, if, you'll, own, they'll, up, we're, they'd, so, our, do, all, him, ours	ourselves, had, nor, before, it, a, she's, as, hadn't, because, has, she, yours, or, above, yourself, herself, she'd, such, they, each, can't, don't, i, until, that, out, he's, cannot, to, we've, hers, you, did, let's, most, here, these, hasn't, was, there, when's, shan't, doing, at, through, been, over, i've, on, being, same, how, whom, my, after, who, itself, me, them, by, then, couldn't, he, should, few, wasn't, again, while, their, not, w...

In [48]:
   val lemmatizedWithDate = nysk.mapPartitions(iter => {
      val pipeline = com.cloudera.datascience.lsa.ParseWikipedia.createNLPPipeline();
      iter.map {
        case (docid, date, text) =>
          (docid.toString, date,
            com.cloudera.datascience.lsa.ParseWikipedia.plainTextToLemmas(text.toLowerCase.split("\\W+").mkString(" "), stopwordsBroadcast, pipeline))
        };
    })

lemmatizedWithDate = MapPartitionsRDD[62] at mapPartitions at <console>:157


MapPartitionsRDD[62] at mapPartitions at <console>:157

In [49]:
    val lemmatized = lemmatizedWithDate.map { case (docid, date, text) => (docid, text) }
    val numTerms = 1000;

lemmatized = MapPartitionsRDD[63] at map at <console>:155
numTerms = 1000


1000

In [50]:
    val (termDocMatrix, termIds, docIds, idfs) = com.cloudera.datascience.lsa.ParseWikipedia.termDocumentMatrix(lemmatized, stopwordsBroadcast, numTerms, sc);
    termDocMatrix.cache();



Adding annotator tokenize
Adding annotator ssplit
Adding annotator pos
Adding annotator lemma
Adding annotator tokenize
Adding annotator ssplit
Adding annotator pos
Adding annotator lemma
Number of terms: 1000


termDocMatrix = MapPartitionsRDD[74] at map at ParseWikipedia.scala:64
termIds = Map(645 -> race, 892 -> tell, 69 -> across, 809 -> fund, 629 -> chance, 365 -> nearly, 138 -> less, 760 -> file, 101 -> claim, 479 -> key, 347 -> within, 846 -> speculation, 909 -> major, 333 -> minister, 628 -> opinion, 249 -> bathroom, 893 -> word, 518 -> loan, 962 -> suite, 468 -> obama, 234 -> german, 941 -> singapore, 0 -> incident, 777 -> prison, 555 -> north, 666 -> management, 88 -> guinea, 481 -> now, 352 -> criminal, 408 -> strong, 977 -> month, 170 -> decide, 523 -> believe, 582 -> wall, 762 -> control, 115 -> step, 683 -> appearance, 730 -> nature, 217 -> world, 276 -> prepare, 994 -> decline, 308 -...


Map(645 -> race, 892 -> tell, 69 -> across, 809 -> fund, 629 -> chance, 365 -> nearly, 138 -> less, 760 -> file, 101 -> claim, 479 -> key, 347 -> within, 846 -> speculation, 909 -> major, 333 -> minister, 628 -> opinion, 249 -> bathroom, 893 -> word, 518 -> loan, 962 -> suite, 468 -> obama, 234 -> german, 941 -> singapore, 0 -> incident, 777 -> prison, 555 -> north, 666 -> management, 88 -> guinea, 481 -> now, 352 -> criminal, 408 -> strong, 977 -> month, 170 -> decide, 523 -> believe, 582 -> wall, 762 -> control, 115 -> step, 683 -> appearance, 730 -> nature, 217 -> world, 276 -> prepare, 994 -> decline, 308 -...

In [59]:
val termIdsTxT = termIds.map(l => l.toString.filter(c => c != '[' & c != ']'))
val outputtermIds= "data/termIds.txt"
termIdsTxT.collect()

Name: Unknown Error
Message: <console>:159: error: not enough arguments for method collect: (pf: PartialFunction[String,B])(implicit bf: scala.collection.generic.CanBuildFrom[Iterable[String],B,That])That.
Unspecified value parameter pf.
       termIdsTxT.collect()
                         ^

StackTrace: 

In [1]:
val mat = new RowMatrix(termDocMatrix)
val k = 10 // nombre de valeurs singuliÃ¨res Ã  garder
val svd = mat.computeSVD(k, computeU=true)
val projections = mat.multiply(svd.V)
val projectionsTxt = projections.rows.map(l => l.toString.filter(c => c != '[' & c != ']'))
// Delete the existing path, ignore any exceptions thrown if the path doesn't exist
val outputProjection = "data/projection_LSA.txt"

projectionsTxt.saveAsTextFile(outputProjection)

Name: Compile Error
Message: <console>:25: error: not found: type RowMatrix
       val mat = new RowMatrix(termDocMatrix)
                     ^
<console>:25: error: not found: value termDocMatrix
       val mat = new RowMatrix(termDocMatrix)
                               ^

StackTrace: 

In [63]:
        val nbClusters = 10
        val nbIterations = 1000
        val runs = 10
        val clustering = KMeans.train(termDocMatrix, nbClusters, nbIterations, runs, "k-means||", 0)
        /*val outputClustering = "hdfs://head.local:9000/user/emeric/clusters"
        try { hdfs.delete(new org.apache.hadoop.fs.Path(outputClustering), true) } 
        catch { case _ : Throwable => { } }
        clustering.save(sc, outputClustering)*/
        
        val classes = clustering.predict(termDocMatrix)
        val outputClasses = "data/classes_LSA.txt"
    
        classes.saveAsTextFile(outputClasses)

nbClusters = 10
nbIterations = 1000
runs = 10
clustering = org.apache.spark.mllib.clustering.KMeansModel@669cb1c2
classes = MapPartitionsRDD[146] at map at KMeansModel.scala:85
outputClasses = data/classes_LSA.txt


data/classes_LSA.txt

In [34]:
 
        val outputData = lemmatizedWithDate.zip(classes).map { case ((docid, date, title),cl) => (docid, date, cl) }.sortBy(_._2).map(l => l.toString.filter(c => c != '(' & c != ')'))
        val outputDataFile = "data/output_LSA.txt"
        //try { hdfs.delete(new org.apache.hadoop.fs.Path(outputDataFile), true) } 
        //catch { case _ : Throwable => { } }
        outputData.saveAsTextFile(outputDataFile)

Adding annotator tokenize
Adding annotator ssplit
Adding annotator pos
Adding annotator lemma
Adding annotator tokenize
Adding annotator ssplit
Adding annotator pos
Adding annotator lemma
Adding annotator tokenize
Adding annotator ssplit
Adding annotator pos
Adding annotator lemma
Adding annotator tokenize
Adding annotator ssplit
Adding annotator pos
Adding annotator lemma


outputData = MapPartitionsRDD[161] at map at <console>:196
outputDataFile = output/output_LSA.txt


output/output_LSA.txt

In [35]:
       
        clustering.clusterCenters.foreach(clusterCenter => {
            val highest = clusterCenter.toArray.zipWithIndex.sortBy(-_._1).map(v => v._2).take(10)
            println("*****")
            highest.foreach { s => print( termIds(s) + "," ) }
            println ()
            }
       )

*****
comment,news,woman,obama,man,dsk,sarkozy,know,article,france,
*****
lagarde,european,minister,finance,candidate,europe,bank,economy,imf,emerge,
*****
percent,stock,index,market,price,fall,higher,euro,investor,rise,
*****
opinion,obama,geithner,scandal,politics,limit,tax,another,previous,rich,
*****
reuter,news,site,copyright,comment,mobile,com,index,capital,review,
*****
bail,court,lawyer,manhattan,apartment,judge,release,prosecutor,jail,attorney,
*****
building,sunday,sinclair,apartment,million,stand,manhattan,bond,outside,suite,
*****
request,information,find,incident,serious,rate,michael,eye,widely,secretary,
*****
com,search,email,update,alert,quote,address,rss,feed,london,
*****
please,person,don,anyone,language,clean,another,tweet,conduct,avoid,


In [40]:
import org.apache.spark.mllib.feature.Word2VecModel
// lire le Word2VecModel
val w2vModel = Word2VecModel.load(sc, "w2vModel")

w2vModel = org.apache.spark.mllib.feature.Word2VecModel@78c028b5


<console>:6: error: Symbol 'type scala.AnyRef' is missing from the classpath.
This symbol is required by 'class org.apache.spark.sql.catalyst.QualifiedTableName'.
Make sure that type AnyRef is in your classpath and check for conflicting dependencies with `-Ylog-classpath`.
A full rebuild may help if 'QualifiedTableName.class' was compiled against an incompatible version of scala.
  lazy val $print: String =  {
           ^


org.apache.spark.mllib.feature.Word2VecModel@78c028b5

In [ ]:
val vectors = w2vModel.getVectors.mapValues(vv => org.apache.spark.mllib.linalg.Vectors.dense(vv.map(_.toDouble))).map(identity)
        // transmettre la map aux noeuds de calcul
val bVectors = sc.broadcast(vectors)

In [37]:

        val idTerms = termIds.map(_.swap)

        val pairs = termDocMatrix.zip(lemmatized);

        var w2vecRepr = pairs.map({ case (row, (docid, lemmas)) => 
          var vSum = Vectors.zeros(100)
          var totalWeight: Double = 0
          val words = lemmas.toSet
          words.foreach { word =>
               val colId = idTerms.get(word);
               if (colId != None) {
                   var weight = row(colId.get);
                   if (! weightTfIdf) {
                       weight = 1
                   }
                   val wvec = bVectors2.value.get(word)
                   if (wvec != None) {          
                       vSum = add(scalarMultiply(weight, wvec.get), vSum)
                       totalWeight += weight
                   }
               }
          }
          if (totalWeight > 0) {
               vSum = scalarMultiply(1.0 / totalWeight, vSum)
          }
          else {
               vSum = Vectors.zeros(100);
          }
          (docid, vSum)
        })/*.filter(vec => Vectors.norm(vec._2, 1.0) > 0.0)*/.persist() 

Name: Unknown Error
Message: lastException: Throwable = null
<console>:216: error: not found: value bVectors2
                          val wvec = bVectors2.value.get(word)
                                     ^

StackTrace: 

In [38]:
val matRDD = w2vecRepr.map{v => v._2}.cache()
        val mat = new RowMatrix(matRDD)
        val matrixTxt = mat.rows.map(l => l.toString.filter(c => c != '[' & c != ']'))
        // Delete the existing path, ignore any exceptions thrown if the path doesn't exist
        val outputMatrix = "data/matrice_W2V.txt"
        //try { hdfs.delete(new org.apache.hadoop.fs.Path(outputMatrix), true) } 
        //catch { case _ : Throwable => { } }
        matrixTxt.saveAsTextFile(outputMatrix)

Name: Unknown Error
Message: <console>:194: error: not found: value w2vecRepr
       val matRDD = w2vecRepr.map{v => v._2}.cache()
                    ^

StackTrace: 

In [ ]:
        val centRed = new StandardScaler(withMean = true, withStd = true).fit(matRDD)
        val matCR: RowMatrix = new RowMatrix(centRed.transform(matRDD))

        val matCompPrinc = matCR.computePrincipalComponents(10)
        val projections = matCR.multiply(matCompPrinc)
        //val matSummary = projections.computeColumnSummaryStatistics()
        val projectionsTxt = projections.rows.map(l => l.toString.filter(c => c != '[' & c != ']'))
        // Delete the existing path, ignore any exceptions thrown if the path doesn't exist
        val outputProjection = "data/projection_W2V.txt"
        //try { hdfs.delete(new org.apache.hadoop.fs.Path(outputProjection), true) } 
        //catch { case _ : Throwable => { } }
        projectionsTxt.saveAsTextFile(outputProjection)

In [ ]:
     val nbClusters = 10
        val nbIterations = 1000
        val runs = 10
        val clustering = KMeans.train(matRDD, nbClusters, nbIterations, runs, "k-means||", 0)
        /*val outputClustering = "hdfs://head.local:9000/user/emeric/clusters"
        try { hdfs.delete(new org.apache.hadoop.fs.Path(outputClustering), true) } 
        catch { case _ : Throwable => { } }
        clustering.save(sc, outputClustering)*/
        
        val classes = clustering.predict(matRDD)
        val outputClasses = "data/classes_W2V.txt"
        //try { hdfs.delete(new org.apache.hadoop.fs.Path(outputClasses), true) } 
        //catch { case _ : Throwable => { } }
        classes.saveAsTextFile(outputClasses)